## 核心概念

这份教程会介绍文档、语料库、向量、模型的基础概念以及与之有关的术语，用于理解和使用gensim。

In [1]:
import pprint

gensim核心的概念有：
1. 文档（Document）：一些文本
2. 语料库（Corpus）：文档的集合
3. 向量（Vector）：文档的数学表征
4. 模型（Model）：用于把向量从一种表征方法转移为另一种方法的算法

### 文档
在gensim中，文档是一种文本序列类型的对象。文档可以是140个字符的推特，也可以是段落、新闻或者是一本书。

In [2]:
document = "Human machine interface for lab abc computer applications"

### 语料库
语料库是文档对象的集合，语料库在gensim中扮演了两种角色：
1. 作为训练模型的输入，在训练期间，模型使用训练语料库去查找常见的主题和话题，初始化模型参数；
2. 组织文档，在训练之后，模型可以用于从新文档中提取信息。

以下是一个实例语料库，包含了9个文档，每个文档仅包含一个句子：

In [3]:
text_corpus = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

在收集语料库之后，还需要进行一些预处理，为了方便起见，在下面的例子中仅去除常见的英文单词（the, an等）以及在语料库中仅出现一次的单词。

In [4]:
# Create a set of frequent words
stoplist = set('for a of the and to in'.split(' '))
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in text_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] 
                     for text in texts]
pprint.pprint(processed_corpus)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


在继续处理之前，需要将语料库中每个单词绑定一个唯一的ID号，通过使用*gensim.corpora.Dictionary*类可以实现这个操作：

In [5]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


由于我们的语料库比较小，只有12个单独的token，对于大型语料库，字典包含成千上万个token是很常见的。

### 向量
为了对文档进行数学上的操作，需要对语料库进行另一种方式的表征。一个方法是使用特征向量表征每个文档。比如说，可以将一个特征看作是“问题-答案”对：
1. 单词在文档中出现了多少次：0次
2. 文档包含了多少个段落：2个
3. 文档使用了多少个字体：5个

问题通常由整数id表示（1，2，3等），因此文档的表征变成了一系列数字对：(1, 0.0), (2, 2.0), (3, 5.0)（对应三个问题的答案）。这就是通常所说的稠密向量，因为向量包含了所有问题的明确的答案。

如果事先知道所有问题，就可以隐含问题，简单地将文档表征为（0, 2, 5)。在gensim中，只允许浮点类型的数字。在实际中，向量经常包含大量的数字0，为了解决内存，gensim删除了所有值为0的向量元素。上面的例子因此变成了(2, 2.0), (3, 5.0).这就是通常所说的稀疏向量或者词袋向量。

如果问题一样，可以比较两个向量。比如说，有两个向量(0.0, 2.0, 5.0)和(0.1, 1.9, 4.9)。两个向量十分相似。

另一种表征文档的方法是使用词袋模型，每个文档可以表示为每个单词的出现频率。例如，词典\['coffee', 'milk', 'sugar', 'spoon'\],某个文档包含字符串"coffee milk coffee",可以表示为向量\[2, 1, 0, 0\]。词袋模型完全忽略了文档token中的顺序，这也是名字“词袋”的来源。

查看token对应的id：

In [6]:
pprint.pprint(dictionary.token2id)

{'computer': 0,
 'eps': 8,
 'graph': 10,
 'human': 1,
 'interface': 2,
 'minors': 11,
 'response': 3,
 'survey': 4,
 'system': 5,
 'time': 6,
 'trees': 9,
 'user': 7}


例如，将一字符串“human computer interaction”向量化，可以使用doc2bow制造文档的词袋模型，该方法将返回词语的稀疏表征：

In [7]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(0, 1), (1, 1)]


每个元组的第一个元素表示了token的id（computer的id是0，human的id是1），第二个元素代表了token的出现次数。

注意到interaion并没有出现在语料库中，所以向量化的结果中也不包括该单词。

可以将整个原始的语料库转换为向量列表：

In [8]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
pprint.pprint(bow_corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]


### 模型

语料库向量化之后，可以开始使用模型。可以将模型视为在两个向量空间进行转换的算法。

一个简单的模型例子是TF-IDF，将词袋表征向量空间转换为每个单词的次数频率（由相对稀有度计算而来）。

以下将会初始化TF-IDF模型，在语料库上训练并且转换字符串“system minors”：

In [9]:
from gensim import models

# use bow_corpus to train tfidf model
tfidf = models.TfidfModel(bow_corpus)
words = "system minors".lower().split()
# 向量化
vecs = dictionary.doc2bow(words)
print(tfidf[vecs])

[(5, 0.5898341626740045), (11, 0.8075244024440723)]


元组的第一个元素：id 5 代表system, id 11 代表minors；元组的第二个元素代表出现频率以及相对稀有度。

一旦创建好模型之后，可以将整个语料库通过tfidf进行索引，为相似度查询做准备：

In [10]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=12)
query_document = "system engineering".split()
# 向量化
query_bow = dictionary.doc2bow(query_document)
# 逐个查询每个文档的相似度
sims = index[tfidf[query_bow]]
print(list(enumerate(sims)))

[(0, 0.0), (1, 0.32448703), (2, 0.41707572), (3, 0.7184812), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]


文档3拥有72%的相似度，文档2拥有42%的相似度，etc。

## 语料库和向量空间

### 语料库格式

gensim提供了很多保存语料库的方法，比如，将语料库保存为Market Matrix 格式：

In [11]:
from gensim import corpora

_corpus = [[(1, 0.5)], []]
corpora.MmCorpus.serialize('~/corpus.mm', _corpus)

除了Market Matrix格式外，还有svmlight, lda-c, low等格式。

## Topics and Transformations

### 创建transformation

首先初始化tfidf模型：

In [14]:
dictionary = corpora.Dictionary(processed_corpus)
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus)

### Transformation 向量

创建完毕后，tfidf将被视作只读对象，可以将任何向量从以前的表征方法（比如词袋）转换为tfidf表示：

In [15]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


transformation还可以被序列化，一个接着一个形成一个链,将tfidf语料通过Latent Semantic Indexing模型转换为2D空间

In [17]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
corpus_lsi = lsi_model[corpus_tfidf]

lsi_model.print_topics(2)

[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '0.460*"system" + 0.373*"user" + 0.332*"eps" + 0.328*"interface" + 0.320*"response" + 0.320*"time" + 0.293*"computer" + 0.280*"human" + 0.171*"survey" + -0.161*"trees"')]

从结果来看，"trees","graph","minors"是有关的单词。

模型可以通过save() load()函数来实现持久化：

In [18]:
import os
import tempfile

with tempfile.NamedTemporaryFile(prefix='model-', suffix='.lsi', delete=False) as temp:
    lsi_model.save(temp.name)
    
loaded_lsi_model = models.LsiModel.load(temp.name)

os.unlink(temp.name)

### 可用的transformation

1. tfidf：
    ```model = models.TfidfModel(corpus, normalize=True)```
2. lsi(latent semantic indexing):
    ```model = models.LsiModel(tfidf_corpus, id2word=dictionary, num_topics=300)```
3. RP(random projections):
    ```model = models.RpModel(tfidf_corpus, num_topics=300)```
4. LDA(Latent Dirichlet Allocation):
    ```model = models.LdaModel(corpus, id2word=dictionary, num_topics=300)```
5. HDP(Hierachical Dirichilet Process):
    ```model = models.HdpModel(corpus, id2word=dictionary)```

## Word2Vec 模型

词袋模型统计文档中单词出现的次数，并将各个单词的次数按照任意顺序作为向量元素存储在向量中，词袋模型效率高，但是丢失了单词出现顺序的信息以及不能反映语义上的区别。

Word2Vec模型在低维向量空间中表征单词，并且在空间中接近的单词也具有相似的含义。

使用google新闻数据集：

In [25]:
import gensim.downloader as api
wv = api.load("word2vec-google-news-300")

In [23]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/3000000 is </s>
word #1/3000000 is in
word #2/3000000 is for
word #3/3000000 is that
word #4/3000000 is is
word #5/3000000 is on
word #6/3000000 is ##
word #7/3000000 is The
word #8/3000000 is with
word #9/3000000 is said


word2vec的缺点是，无法infer在语料库中没有出现的单词。

更进一步，word2vec支持多种单词相似度的任务：

In [29]:
pairs = [
    ('car', 'minivan'),
    ('car', 'bicycle'),
    ('car', 'airplane'),
    ('car', 'cereal'),
    ('car', 'communism')
]

for w1, w2, in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.69
'car'	'bicycle'	0.54
'car'	'airplane'	0.42
'car'	'cereal'	0.14
'car'	'communism'	0.06


In [30]:
# 输出相似度最高的n个单词
print(wv.most_similar(positive=['car', 'minivan'], topn=5))

[('SUV', 0.8532191514968872), ('vehicle', 0.8175783753395081), ('pickup_truck', 0.7763689160346985), ('Jeep', 0.7567334175109863), ('Ford_Explorer', 0.7565719485282898)]
